<a href="https://colab.research.google.com/github/yeaeunJi/deep_learning-/blob/main/%ED%95%99%EC%8A%B5%EA%B4%80%EB%A0%A8%EA%B8%B0%EC%88%A0%EB%93%A4(1)_%EB%A7%A4%EA%B0%9C%EB%B3%80%EC%88%98%EA%B0%B1%EC%8B%A0.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## 매개변수 갱신
- 신경망의 목적 : 손실 함수의 값을 가능한 한 낮추는 매개변수 찾기

   ==> 최적화(optimization)

- 확률적 경사 하강법(SGD) : 최적의 매개변수 값을 찾는 방법 중 하나로 매개변수의 기울기(미분)을 통해서 기울어진 방향으로 매개변수 값을 갱신을 반복함으로써 최적의 매개변수 값을 찾고자 함

- SGD의 단점과 다른 최적화 기법을 배우고자 함

### 확률적 경사 하강법(SGD)

In [1]:
class SGD :
  def __init__(self, lr = 0.01) :
    self.lr = lr
  
  def update(self, params, grads) :
    for key in params.keys() :
      params[key] -=  self.lr * grads[key] # params['W1'] : 가중치, grads['W1'] : 기울기

# SGD 클래스를 사용한 신경망 매개변수의 진행 예
# network = TwoLayerNet(매개변수)
# optimizer = SGD()

# for i in range(1000) :
#   x_batch, y_batch = get_mini_batch(...) 
#   grads = network.gradient(x_batch, t_batch)
#   params = network.params
#   optimizer.update(params, grads)

### SGD 단점
- 단순하고 구현이 쉽지만 경우에 따라 비효율적일 때가 존재함
  - 비등방성(anisotropy) 함수(방향에 따라 기울기가 달리지는 함수)에서 매개변수의 기울기 탐색 경로가 비효율적(지그재그로 이동)

### 모멘텀(Momentum)
-  '운동량'을 의미하는 단어
- 공이 그릇의 곡면(기울기)를 따라 구르듯 움직이듯이 최적화하는 방법
- SGD와 비교하면 최적화하는 경로의 지그재그 정도가 덜함. 그 이유는 x축의 힘은 아주 작지만 방향을 변하지 않아 한 방향으로 일정하게 가속하기 때문임. y축의 힘은 크지만 위아래로 번갈아 상충하여 속도가 안정적이지 않음
- 속도(v) : av(물체가 아무런 힘을 받지 않을 때 서서히 하강시키는 역할) - 학습률*W(가중치)에 대한 손실 함수의 기울기
- W(가중치) = W + v

In [ ]:
class Momentum :
  def __init__(self, lr=0.1, momentum=0.9) :
    self.lr = lr
    self.momentum = momentum
    self.v = None # 물체의 속도. 

  def update(self, params, grads) :
    if self.v is None :
      self.v = {}  # 매개변수와 같은 구조의 데이터를 dict 변수로 저장
      for key, val in params.items() :
        self.v[key] = np.zeros_like(val)
      
    for key in params.keys() :
      self.v[key] = self.momentum + self.v[key] - self.lr * grads[key]
      params[key] += self.v[key]

### AdaGrad
- 신경망 학습에서는 학습률 값이 중요한데, 학습률이 너무 작으면 학습 시간이 증가하게 되고 학습률이 너무 크면 발산하여 학습이 제대로 이뤄지지 않음

- 학습률을 정하는 기술로는 '학습률 감소(learning rate decay)가 있음
  - 학습을 진행하면서 학습률을 점차 줄여가는 방법으로 처음에는 크게 학습하다가 조금씩 작게 학습하도록 함
  - 학습률을 서서히 낮추는 간단한 방법으로는 매개변수 전체의 학습률을 일괄적으로 낮추는 방법이 있는데, 이를 발전시킨 것이 AdaGrad임

- AdaGrad는 각각의 매개변수에 맞는 값을 만들어줌. 즉, 개별 매개변수에 적응적으로(adaptive) 학습률을 조정하며 학습을 진행
  - 매개변수의 원소 중에서 많이 갱신된 원소를 학습률이 낮아짐
  - 과거의 기울기를 제곱하여 계속하여 더하므로 학습이 진행될 수록 학습 정도가 약해짐. 
  - 무한히 학습하다보면 갱신률이 0이되어 전혀 갱신되지 않게 됨. 이문제를 해결한 기법으로 RMSProp가 있으며 이는 먼 과거의 기울기는 잊고 새로운 기울기 정보를 크게 반영하는 것임. 이를 '지수이동평균(Exponential Moving Average, EMA)라 함. 과거 기울기의 반영 규모를 기하급수적으로 감소시킴

In [ ]:
class AdaGrad :
  def __init__(self, lr = 0.01) :
    self.lr = lr
    self.h = None

  def update(self, params, grads) :
    if self.h is None :
      self.h = {}

      for key, val in  params.items() :
        self.h[key] = np.zeros_like(val)

      for key in params.keys() :
        self.h[key] += grads[key] * grads[key]
        params[key] -= self.lr * grads[key] / (np.sqrt(self.h[key] + 1e-7)) # self.h[key]값이 0이 되더라도 0으로 나누지 않도록 하기 위해 아주 작은 값을 더함

### Adam
- 마치 모멘텀과 AdaGrad를 융합한 듯 보이기도 하는 방법으로 매개 변수 공간을 효율적으로 탐색 및 하이퍼파라미터의 '편향 보정'이 진행되는 것이 특징임
- 그릇 바닥을 구르듯 움직이며 최적화를 진행. 모멘텀과 비슷한 패턴이지만 그보다는 공의 좌우 흔들림이 적음.
- 논문에 따르면 Adam의 하이퍼파라미터는 학습률, 일차 모멘텀용 계수 베타1, 이차 모멘텀용 계수 베타2 3가지이다. 기본 설정값은 베타1 = 0.9, 베타2 = 0.999d이면 많은 경우 좋은 결과를 얻을 수 있다고 함

In [ ]:
# 아래 코드는 https://github.com/WegraLee/deep-learning-from-scratch/blob/master/common/optimizer.py에서 가져온 소스 코드이다
class RMSprop:

    """RMSprop"""

    def __init__(self, lr=0.01, decay_rate = 0.99):
        self.lr = lr
        self.decay_rate = decay_rate
        self.h = None
        
    def update(self, params, grads):
        if self.h is None:
            self.h = {}
            for key, val in params.items():
                self.h[key] = np.zeros_like(val)
            
        for key in params.keys():
            self.h[key] *= self.decay_rate
            self.h[key] += (1 - self.decay_rate) * grads[key] * grads[key]
            params[key] -= self.lr * grads[key] / (np.sqrt(self.h[key]) + 1e-7)


class Adam:

    """Adam (http://arxiv.org/abs/1412.6980v8)"""

    def __init__(self, lr=0.001, beta1=0.9, beta2=0.999):
        self.lr = lr
        self.beta1 = beta1
        self.beta2 = beta2
        self.iter = 0
        self.m = None
        self.v = None
        
    def update(self, params, grads):
        if self.m is None:
            self.m, self.v = {}, {}
            for key, val in params.items():
                self.m[key] = np.zeros_like(val)
                self.v[key] = np.zeros_like(val)
        
        self.iter += 1
        lr_t  = self.lr * np.sqrt(1.0 - self.beta2**self.iter) / (1.0 - self.beta1**self.iter)         
        
        for key in params.keys():
            #self.m[key] = self.beta1*self.m[key] + (1-self.beta1)*grads[key]
            #self.v[key] = self.beta2*self.v[key] + (1-self.beta2)*(grads[key]**2)
            self.m[key] += (1 - self.beta1) * (grads[key] - self.m[key])
            self.v[key] += (1 - self.beta2) * (grads[key]**2 - self.v[key])
            
            params[key] -= lr_t * self.m[key] / (np.sqrt(self.v[key]) + 1e-7)
            
            #unbias_m += (1 - self.beta1) * (grads[key] - self.m[key]) # correct bias
            #unbisa_b += (1 - self.beta2) * (grads[key]*grads[key] - self.v[key]) # correct bias
            #params[key] += self.lr * unbias_m / (np.sqrt(unbisa_b) + 1e-7)


### 경우에 따라 이 4가지 기법이 좋은 결과를 낼 때가 다름. 하지만 일반적으로 SGD보다 다른 기법들이 빠르게 학습하고, 때로 최종 정확도도 높게 나타난다고 함.
많은 연구에서 SGD를 사용하고 있음